# Logistic Regression Consulting Project

## Binary Customer Churn

Create a machine learning model that will help predict which customers will churn (stop buying their service).

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [13]:
from pyspark.sql import SparkSession

In [14]:
spark = SparkSession.builder.appName('logregproject').getOrCreate()

In [15]:
df = spark.read.csv('9.3.customer_churn.csv', header=True, inferSchema=True)

In [16]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [7]:
df.head(5)

[Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1),
 Row(Names='Kevin Mueller', Age=41.0, Total_Purchase=11916.22, Account_Manager=0, Years=6.5, Num_Sites=11.0, Onboard_date=datetime.datetime(2013, 8, 13, 0, 38, 46), Location='6157 Frank Gardens Suite 019 Carloshaven, RI 17756', Company='Wilson PLC', Churn=1),
 Row(Names='Eric Lozano', Age=38.0, Total_Purchase=12884.75, Account_Manager=0, Years=6.67, Num_Sites=12.0, Onboard_date=datetime.datetime(2016, 6, 29, 6, 20, 7), Location='1331 Keith Court Alyssahaven, DE 90114', Company='Miller, Johnson and Wallace', Churn=1),
 Row(Names='Phillip White', Age=42.0, Total_Purchase=8010.76, Account_Manager=0, Years=6.71, Num_Sites=10.0, Onboard_date=datetime.datetime(2014, 4, 22, 12, 43, 12), Location='13120 Daniel Mount Angelabury, WY 30645-4695'

In [18]:
df.describe(['Age', 'Total_Purchase', 'Account_Manager' ,'Years', 'Num_Sites', 'Churn']).show()

+-------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+
|summary|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|              Churn|
+-------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+
|  count|              900|              900|               900|              900|               900|                900|
|   mean|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|0.16666666666666666|
| stddev|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| 0.3728852122772358|
|    min|             22.0|            100.0|                 0|              1.0|               3.0|                  0|
|    max|             65.0|         18026.01|                 1|             9.15|              14.0|                  1|
+-------+---------------

Drop columns: 
* Names
* Onboard_date
* Location

What are the unique values in `Company` column? 

In [9]:
df.groupBy('Company').count().show() # Too many to keep as a categorical variable

+--------------------+-----+
|             Company|count|
+--------------------+-----+
|Miller, Johnson a...|    1|
|Hunter, Reyes and...|    1|
|          Obrien PLC|    1|
|            Soto PLC|    2|
|            Todd LLC|    1|
|Smith, Marshall a...|    1|
|           Smith PLC|    1|
|          Hall Group|    1|
|Freeman, Lam and ...|    1|
|       Smith-Carroll|    1|
|Hall, Hernandez a...|    1|
|          Cannon Inc|    1|
|        White-Dennis|    1|
|Wilson, Collins a...|    1|
|Jennings, Gates a...|    1|
|     Campbell-Willis|    1|
|    Martinez-Roberts|    1|
|        Robinson PLC|    1|
|          Barton Inc|    1|
|Hernandez, Middle...|    1|
+--------------------+-----+
only showing top 20 rows



In [20]:
df = df.select(['Age', 'Total_Purchase', 'Account_Manager', 'Years',
               'Num_Sites', 'Churn']).na.drop()

In [19]:
from pyspark.ml.feature import VectorAssembler

In [22]:
assembler = VectorAssembler(inputCols = ['Age', 'Total_Purchase', 'Account_Manager', 'Years','Num_Sites'],
                           outputCol = 'features')

In [23]:
df_final = assembler.transform(df).select(['features', 'churn'])

#### Train-test split

In [24]:
df_train, df_test = df_final.randomSplit([0.7, 0.3])

#### Fit the model

In [11]:
from pyspark.ml.classification import LogisticRegression

In [26]:
log_reg = LogisticRegression(featuresCol='features', labelCol='churn').fit(df_train)

#### Evaluate model

In [27]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [30]:
results = log_reg.evaluate(df_test)
results.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8939.61,0.0...|    0|[6.57222106259870...|[0.99860326620654...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.47382101848123...|[0.81363746741713...|       0.0|
|[28.0,11128.95,1....|    0|[4.11993957064344...|[0.98401420111652...|       0.0|
|[28.0,11204.23,0....|    0|[1.91302528853749...|[0.87135864194435...|       0.0|
|[29.0,8688.17,1.0...|    1|[2.68364769246941...|[0.93605480821618...|       0.0|
|[29.0,9378.24,0.0...|    0|[4.92529146926706...|[0.99279172673222...|       0.0|
|[29.0,11274.46,1....|    0|[4.43990357898604...|[0.98834047238192...|       0.0|
|[29.0,13240.01,1....|    0|[6.65662281532263...|[0.99871616998577...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.71109032741632...|[0.97613272582286...|       0.0|
|[30.0,8403.78,1

In [28]:
eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='churn')

In [31]:
eval.evaluate(results.predictions)

0.7149369941604343

#### Predict on holdout data

In [32]:
df_new_customers = spark.read.csv('9.4.new_customers.csv', header=True, inferSchema=True)

In [33]:
df_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [34]:
df_new_test = assembler.transform(df_new_customers)

In [35]:
df_new_test

DataFrame[Names: string, Age: double, Total_Purchase: double, Account_Manager: int, Years: double, Num_Sites: double, Onboard_date: timestamp, Location: string, Company: string, features: vector]

In [36]:
final_results = log_reg.transform(df_new_test)

In [37]:
final_results.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|            features|       rawPrediction|         probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,1.0...|[2.20771177061235...|[0.90093989603208...|       0.0|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,1.0...|[-6.2123952258813...|[0.00200042094046...|       

In [39]:
final_results.select(['Company', 'prediction']).show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



Ok! That is it! Now we know that we should assign Account Managers to Cannon-Benson,Barron-Robertson,Sexton-GOlden, and Parks-Robbins!